In [118]:
import numpy as np
import pandas as pd
import pickle as pkl
import numbers

/anaconda3/envs/NLP/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (6,9,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1008,1009,1010,1011,1014,1015,1016,1017,1018,1019,1021,1022,1023,1027,1028,1029,1030,1031,1032,1034,1035,1036,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1065,1066,1067,1068,1069,1070,1071,1073,1074,1075,1076,1078,1079,1080,1081,1082,1083,1084,1086,1087,1088,1089,1091,1092,1093,1094,1095,1096,1097,1099,1100,1101,1102,1104,1105,1106,1107,1108,1109,1110,1112,1113,1114,1115,1118,1119,1121,1122,1123,1125,1127,1128,1131,1132,1134,1135,1136,1138,1140,1141,1144,1145,1146,1147,1148,1149,1150,1151,1152,1153,1154,1157,1158,1159,1160,1161,1162,1163,1164,1165,1166,1167,1170,1171,1172,1173,1174,1175,1177,1178,1179,1180,1183,1184,1185,1186,1187,1188,1190,1192,1196,1199,1200,1201,1209,1212,1213,1214,1222,1223,1224,1225,1226,1227,1235,1236

In [82]:
data16_17['MN_EARN_WNE_INC1_P10'].describe()
data16_17['MD_EARN_WNE_P6'].describe()

count                  6091
unique                  528
top       PrivacySuppressed
freq                    562
Name: MD_EARN_WNE_P6, dtype: object

In [141]:
cost_a = list(data16_17['COSTT4_A'])
cost_p = list(data16_17['COSTT4_P'])
cost = []
for i in range(len(cost_a)):
    if not np.isnan(cost_a[i]):
        if not np.isnan(cost_p[i]):
            raise Exception('No school is like this')
        cost.append(cost_a[i])
    else:
        cost.append(cost_p[i])

earning = data16_17['MD_EARN_WNE_P6']
cost_earning_df = pd.DataFrame({'cost':cost, 'earning':earning})

In [142]:
features = []

admin_features = ['ADM_RATE', 'SAT_AVG']
repay_features = ['COMPL_RPY_1YR_RT', 'COMPL_RPY_3YR_RT', 'COMPL_RPY_5YR_RT', 'COMPL_RPY_7YR_RT', 'RPY_1YR_N', 'RPY_3YR_N', 'RPY_5YR_N', 'RPY_7YR_N']
complete_features = ['D100_4', 'D100_L4', 'C100_4', 'C100_L4']
aid_features = ['GRAD_DEBT_MDN_SUPP', 'DEBT_MDN_SUPP', 'CUML_DEBT_N', 'GRAD_DEBT_MDN', 'DEBT_N', 'GRAD_DEBT_N']
root_features = ['LATITUDE', 'LONGITUDE']
school_features = ['REGION', 'LOCALE', 'ACCREDAGENCY', 'SCH_DEG', 'MAIN', 'NUMBRANCH', 
                   'HIGHDEG', 'CONTROL', 'ST_FIPS', 'AVGFACSAL', 'PFTFAC','ICLEVEL']
student_features = ['UG', 'UG_NRA', 'UG_UNKN', 'UG_WHITENH', 'UG_BLACKNH', 'UG_API', 'UG_AIANOLD', 
                    'UG_HISPOLD', 'APPL_SCH_PCT_GE2', 'APPL_SCH_PCT_GE3', 'APPL_SCH_PCT_GE4', 'APPL_SCH_PCT_GE5', 
                    'MEDIAN_HH_INC', 'UNEMP_RATE', 'UGDS_MEN', 'UGDS_WOMEN','UGNONDS','GRADS']

features.extend(admin_features)
features.extend(repay_features)
features.extend(complete_features)
features.extend(aid_features)
features.extend(root_features)
features.extend(school_features)
features.extend(student_features)

# We drop these variables because there are too many missing values in those variables.
features_drop = ['ADM_RATE', 'SAT_AVG', 'RPY_1YR_N', 'RPY_3YR_N', 'RPY_5YR_N', 'RPY_7YR_N',
                'D100_4', 'D100_L4', 'APPL_SCH_PCT_GE2', 'APPL_SCH_PCT_GE3', 'APPL_SCH_PCT_GE4', 
                 'APPL_SCH_PCT_GE5', 'UGNONDS','GRADS', 'GRAD_DEBT_MDN_SUPP', 'CUML_DEBT_N', 'ACCREDAGENCY']

ROI = data16_17[features]
ROI = ROI.drop(features_drop, 1)
ROI['cost'] = cost_earning_df['cost']
ROI['earning'] = cost_earning_df['earning']

ROI.to_csv('./ROI.csv')

In [143]:
def normalizeColumn(column, data):
    if isinstance(data[column][0], numbers.Number):
        return [float(i)/sum(data[column].values) for i in data[column].values]
    else:
        return data[column]

def normalizeData(data):
    for column in data.columns:
        data[column] = normalizeColumn(column, data)
    return data

def cleanColumn(column, data):
    if type(data[column][0]) == type('.'):
        data[column][data[column] == 'PrivacySuppressed'] = np.nan
    return data[column]

def cleanData(data):
    for column in data.columns:
        data[column] = cleanColumn(column, data)
    return data

np.sum(ROI.isna())

COMPL_RPY_1YR_RT    1092
COMPL_RPY_3YR_RT     893
COMPL_RPY_5YR_RT    1012
COMPL_RPY_7YR_RT    1295
C100_4              5028
C100_L4             3590
DEBT_MDN_SUPP         19
GRAD_DEBT_MDN         19
DEBT_N                19
GRAD_DEBT_N           19
LATITUDE             438
LONGITUDE            438
REGION                 0
LOCALE               438
SCH_DEG              308
MAIN                   0
NUMBRANCH              0
HIGHDEG                0
CONTROL                0
ST_FIPS                0
AVGFACSAL           2908
PFTFAC              3421
ICLEVEL                0
UG                  2705
UG_NRA              2705
UG_UNKN             2705
UG_WHITENH          2705
UG_BLACKNH          2705
UG_API              2705
UG_AIANOLD          2705
UG_HISPOLD          2705
MEDIAN_HH_INC       2033
UNEMP_RATE          2033
UGDS_MEN             728
UGDS_WOMEN           728
cost                1203
earning             1084
dtype: int64

In [144]:
ROI = cleanData(ROI)
ROI

/anaconda3/envs/NLP/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,COMPL_RPY_1YR_RT,COMPL_RPY_3YR_RT,COMPL_RPY_5YR_RT,COMPL_RPY_7YR_RT,C100_4,C100_L4,DEBT_MDN_SUPP,GRAD_DEBT_MDN,DEBT_N,GRAD_DEBT_N,...,UG_BLACKNH,UG_API,UG_AIANOLD,UG_HISPOLD,MEDIAN_HH_INC,UNEMP_RATE,UGDS_MEN,UGDS_WOMEN,cost,earning
0,0.3366762178,0.429326288,0.5113182423,0.5638709677,0.0643,NaN,13246,32750,3085,795,...,0.9055,0.0030,0.0027,0.0027,49720.22,4.84000015258789,0.4283,0.5717,22667.0,25200
1,0.5745573159,0.6182700794,0.6733300829,0.7405929304,0.3018,NaN,14979.5,21833,6302,2952,...,0.2861,0.0286,0.0031,0.0101,55735.22,3.45000004768371,0.4070,0.5930,22684.0,35100
2,0.3333333333,0.4390243902,0.5507246377,0.679245283,0.1429,NaN,10430,22890,398,33,...,0.2347,0.0204,0.0000,0.0612,53683.7,3.59999990463256,0.3891,0.6109,13380.0,30700
3,0.6408888889,0.680887372,0.742248062,0.816953317,0.1533,NaN,13522,22647,2626,1219,...,0.1435,0.0366,0.0205,0.0186,58688.62,3.64000010490417,0.5830,0.4170,22059.0,36200
4,0.2446808511,0.3660377358,0.4423076923,0.4919678715,0.0725,NaN,15738,31500,3486,1031,...,0.9172,0.0018,0.0005,0.0028,46065.2,4.80999994277954,0.3788,0.6212,19242.0,22600
5,0.6572383646,0.7118012422,0.7585524456,0.8045007032,0.4385,NaN,17340,23290,10443,5918,...,0.1450,0.0105,0.0061,0.0086,57928.41,3.25999999046325,0.4452,0.5548,28422.0,37400
6,0.2702702703,0.325,0.53125,0.5,NaN,0.1411,5764,9500,851,172,...,0.2080,0.0038,0.0051,0.0044,43985.14,3.75999999046325,0.3934,0.6066,13868.0,23100
7,0.5269058296,0.6019845645,0.6093418259,0.7012658228,NaN,NaN,13461,18000,1786,881,...,0.1187,0.0056,0.0165,0.0038,50752.1,3.38000011444091,0.3390,0.6610,NaN,33400
8,0.5849335303,0.634180791,0.701754386,0.7112561175,0.0720,NaN,10750,23363,2522,764,...,0.3189,0.0178,0.0046,0.0112,50723.81,3.78999996185302,0.3556,0.6444,19255.0,30100
9,0.7699652778,0.8109577583,0.8045977011,0.8432835821,0.4682,NaN,17000,21500,5807,3802,...,0.0694,0.0116,0.0047,0.0083,59005.4,3.14000010490417,0.5067,0.4933,29794.0,39500


In [136]:
ROI = pkl.load(open('./data_test.p', 'rb'))
ROI = ROI.dropna()
ROI['earning'] = ROI['earning'].astype('int64')
ROI['cost'] = ROI['cost'].astype('int64')
ROI['ROI'] = (ROI['earning'])/ROI['cost']
ROI = ROI.drop(['cost', 'earning'], 1)
ROI_X = ROI.drop('ROI', 1)
ROI_Y = ROI['ROI']

In [137]:
ROI_X.head()

,COMPL_RPY_1YR_RT,COMPL_RPY_3YR_RT,COMPL_RPY_5YR_RT,COMPL_RPY_7YR_RT,DEBT_MDN_SUPP,GRAD_DEBT_MDN,DEBT_N,GRAD_DEBT_N,LATITUDE,LONGITUDE,NUMBRANCH,REGION,LOCALE,SCH_DEG,MAIN,HIGHDEG,CONTROL,ST_FIPS
0,0.336676,0.429326,0.511318,0.563871,13246.0,32750.0,3085.0,795.0,34.783368,-86.568502,1,5,12.0,3.0,1,4,1,1
1,0.574557,0.618270,0.673330,0.740593,14979.5,21833.0,6302.0,2952.0,33.505697,-86.799345,1,5,12.0,3.0,1,4,1,1
3,0.640889,0.680887,0.742248,0.816953,13522.0,22647.0,2626.0,1219.0,34.724557,-86.640449,1,5,12.0,3.0,1,4,1,1
4,0.244681,0.366038,0.442308,0.491968,15738.0,31500.0,3486.0,1031.0,32.364317,-86.295677,1,5,12.0,3.0,1,4,1,1
5,0.657238,0.711801,0.758552,0.804501,17340.0,23290.0,10443.0,5918.0,33.211875,-87.545978,1,5,13.0,3.0,1,4,1,1


In [138]:
from sklearn.ensemble import GradientBoostingRegressor

gbm = GradientBoostingRegressor()
gbm = gbm.fit(ROI_X, ROI_Y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [140]:
gbm.feature_importances_

array([0.00423971, 0.00983433, 0.01230364, 0.00473148, 0.15045095,
       0.03300511, 0.05287571, 0.01353378, 0.00825013, 0.00742789,
       0.00580745, 0.01377249, 0.00293356, 0.00096597, 0.01721398,
       0.00496738, 0.6536978 , 0.00398863])